# Upload the CSV files 'legislators-historical.csv' from Google Classroom via the 'Files' tab on the left

In [0]:
import os
os.listdir()

['.config', 'legislators-historical.csv', 'sample_data']

# Load the data into a pandas dataframe

In [0]:
import pandas as pd

leg = pd.read_csv('legislators-historical.csv', 
                  usecols=['first_name', 'gender', 'type', 'state', 'party', 'birthday', 'last_name'])

In [0]:
# show 3 rows at random
leg.sample(3)

,last_name,first_name,birthday,gender,type,state,party
3597,Wickliffe,Charles,1788-06-08,M,rep,KY,Unionist
10968,Ritter,Donald,1940-10-21,M,rep,PA,Republican
3157,Warner,Hiram,1802-10-29,M,rep,GA,Democrat


In [0]:
# Count the number of congress members per state
# the result is ordered by the group key by default
state_counts = leg.groupby('state')['last_name'].count()
state_counts.head()

state
AK     16
AL    206
AR    117
AS      2
AZ     48
Name: last_name, dtype: int64

In [0]:
# If we leave the ['last_name'] away, each column is counted independently, 
# leading to a lot of redundancy
leg.groupby('state').count().head()

,last_name,first_name,birthday,gender,type,party
state,,,,,,
AK,16,16,16,16,16,14
AL,206,206,197,206,206,206
AR,117,117,114,117,117,115
AS,2,2,2,2,2,2
AZ,48,48,48,48,48,48


In [0]:
# we can use two group key columns to make the groups more granular:
# here we count the number of persons for each state and each gender
state_gender_counts = leg.groupby(['state', 'gender'])['last_name'].count()
state_gender_counts.head()

state  gender
AK     M          16
AL     F           3
       M         203
AR     F           5
       M         112
Name: last_name, dtype: int64

In [0]:
# we can sort to see the most common combination of state and gender
state_gender_counts.sort_values(ascending=False)

state  gender
NY     M         1441
PA     M         1046
OH     M          666
IL     M          473
VA     M          428
                 ... 
GU     F            1
RI     F            1
VI     F            1
WV     F            1
MT     F            1
Name: last_name, Length: 104, dtype: int64

# Groupby changes the index

In [0]:
# apart from counting the number of rows per group, the groupby also changes the index by default:
# in the original dataframe we had a RangeIndex:
leg.index

RangeIndex(start=0, stop=11975, step=1)

In [0]:
# this means each number is labelled with an increasing number starting from zero
# and we can use these numbers to filter on the rows
leg.loc[0:2]

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN


In [0]:
# recall the dataframe with the number of senators per state from before
state_counts.sample(4)

state
WY     40
KY    373
LA    197
FL    155
Name: last_name, dtype: int64

In [0]:
# by contrast, the grouped dataframe has the state codes as index
state_counts.index

Index(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'DK', 'FL',
       'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD',
       'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OL', 'OR', 'PA', 'PI', 'PR', 'RI', 'SC', 'SD',
       'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype='object', name='state')

In [0]:
# so we can use the state codes to access individual rows
state_counts.loc['AK']

16

In [0]:
state_counts.loc['WY']

40

In [0]:
# numbers cannot be used to access individual rows
state_counts.loc[0]

TypeError: ignored

In [0]:
# recall the dataframe from before where we counted by state AND gender
state_gender_counts.sample(3)

state  gender
SC     F          5
OK     F          2
ID     M         57
Name: last_name, dtype: int64

In [0]:
# grouping by two columns causes 'state' and 'gender' to become a MultiIndex:
state_gender_counts.index

MultiIndex([('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F'),
            ('AR', 'M'),
            ('AS', 'M'),
            ('AZ', 'F'),
            ('AZ', 'M'),
            ('CA', 'F'),
            ('CA', 'M'),
            ...
            ('VI', 'F'),
            ('VI', 'M'),
            ('VT', 'M'),
            ('WA', 'F'),
            ('WA', 'M'),
            ('WI', 'M'),
            ('WV', 'F'),
            ('WV', 'M'),
            ('WY', 'F'),
            ('WY', 'M')],
           names=['state', 'gender'], length=104)

In [0]:
# to access rows via a MultiIndex, provide a tuple of index values to .loc
state_gender_counts.loc[('TX', 'M')]

252

In [0]:
# to get all values on a particular level of a MultiIndex:
state_gender_counts.index.get_level_values('state')

Index(['AK', 'AL', 'AL', 'AR', 'AR', 'AS', 'AZ', 'AZ', 'CA', 'CA',
       ...
       'VI', 'VI', 'VT', 'WA', 'WA', 'WI', 'WV', 'WV', 'WY', 'WY'],
      dtype='object', name='state', length=104)

In [0]:
state_gender_counts.reset_index()

,state,gender,last_name
0,AK,M,16
1,AL,F,3
2,AL,M,203
3,AR,F,5
4,AR,M,112
...,...,...,...
99,WI,M,196
100,WV,F,1
101,WV,M,119
102,WY,F,2


In [0]:
# to sort by an index level, use sort_index(level=...) instead of sort_values(...)
state_gender_counts.sort_index(level='gender')

state  gender
AL     F           3
AR     F           5
AZ     F           3
CA     F          23
CO     F           3
                ... 
VT     M         115
WA     M          90
WI     M         196
WV     M         119
WY     M          38
Name: last_name, Length: 104, dtype: int64

In [0]:
# if you do not wish to change the index of the original ungrouped dataframe
# while grouping, use as_index=False:
without_multiindex = leg.groupby(['state', 'gender'], as_index=False)['last_name'].count()
without_multiindex.head()

,state,gender,last_name
0,AK,M,16
1,AL,F,3
2,AL,M,203
3,AR,F,5
4,AR,M,112


In [0]:
# In this case, the columns used for the grouping remain normal columns and do not get become an index
without_multiindex[['gender', 'last_name']]

,gender,last_name
0,M,16
1,F,3
2,M,203
3,F,5
4,M,112
...,...,...
99,M,196
100,F,1
101,M,119
102,F,2


In [0]:
without_multiindex.loc[('CA', 'M')]

KeyError: ignored

In [0]:
without_multiindex.loc[1]

state        AL
gender        F
last_name     3
Name: 1, dtype: object

# Exercise 1: group by / aggregations

### Exercise 1.1: Count the number of senators per party
The result should be a dataframe with one index column (the party, e.g. "Republican" or "Democrat") and one count column (the number of senators for that specific party)

### Bonus: Sort the result in descending order

In [0]:
#party_counts = <YOUR CODE>

In [0]:
party_counts = leg.groupby('party')['state'].count()
party_counts.head(10)

party
Adams                      99
Adams Democrat              1
American                   44
American Labor              2
Anti Jackson                1
Anti Jacksonian            31
Anti Masonic               27
Anti-Administration         9
Anti-Jacksonian            14
Anti-Lecompton Democrat     7
Name: state, dtype: int64

In [0]:
# Bonus
party_counts.sort_values(ascending=False, inplace=True)
party_counts.head(10)

party
Democrat                    5003
Republican                  4878
Whig                         564
Federalist                   343
Jackson                      338
Adams                         99
Ind. Republican-Democrat      70
American                      44
Populist                      42
Unionist                      42
Name: state, dtype: int64

### Exercise 1.2: First filter by party then group by gender
Filter the records by party == 'Democrat', then count the number of senators for each gender

In [0]:
#democrats_by_gender = <YOUR CODE>

In [0]:
democrats_by_gender = leg[leg['party'] == 'Democrat'].groupby('gender')['state'].count()
democrats_by_gender.head(10)

gender
F     139
M    4864
Name: state, dtype: int64

In [0]:
# this is essentially equivalent to filtering the original dataframe 
# by 'Democrat AND male' and 'Democrat AND female':

print(leg.loc[(leg['party'] == 'Democrat') & (leg['gender'] == 'F'), 'state'].count())
print(leg.loc[(leg['party'] == 'Democrat') & (leg['gender'] == 'M'), 'state'].count())

139
4864


### Exercise 1.3: Group by year of birthday and gender
First, create a new column called 'year' that contains the year (first four digits of the birthday); hint: use leg['birthday'].str.slice(...)

Then do a count grouped by both year and gender. 

### Bonus:
Do not create a MultiIndex, use the original columns

In [0]:
#<YOUR CODE>

In [0]:
leg_with_year.head()

,last_name,first_name,birthday,gender,type,state,party,year
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration,1745
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN,1742
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN,1743
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN,1730
4,Clymer,George,1739-03-16,M,rep,PA,NaN,1739


In [0]:
# don't want to alter the original dataframe, so make a copy
leg_with_year = leg.copy()

# create the new column with the year
leg_with_year['year'] = leg_with_year['birthday'].str.slice(0, 4)

# do the groupby
by_year_and_gender = leg_with_year.groupby(['year', 'gender'])['state'].count()

# inspect the results
by_year_and_gender.head(-10)

year  gender
1721  M          1
1726  M          2
1727  M          2
1728  M          2
1729  M          3
                ..
1970  M         10
1971  F          1
      M          2
1972  M          3
1973  M          4
Name: state, Length: 338, dtype: int64

In [0]:
# Bonus
# to not create a MultiIndex, use the argument as_index=False

by_year_and_gender = leg_with_year.groupby(['year', 'gender'], as_index=False)['state'].count()
by_year_and_gender.head(-10)

,year,gender,state
0,1721,M,1
1,1726,M,2
2,1727,M,2
3,1728,M,2
4,1729,M,3
...,...,...,...
333,1970,M,10
334,1971,F,1
335,1971,M,2
336,1972,M,3


### Exercise 1.4: Show the list of parties present in each state
Hint: Use the aggregation function .unique()

In [0]:
unique_parties_by_state = <YOUR CODE>

In [0]:
unique_parties_by_state = leg.groupby('state')['party'].unique()
unique_parties_by_state.head(10)

state
AK             [Democrat, Independent, nan, Republican]
AL    [Republican, Jackson, Democrat, Whig, American...
AR    [nan, Democrat, Whig, Republican, Liberal Repu...
AS                                           [Democrat]
AZ                  [Republican, Independent, Democrat]
CA    [Democrat, Independent, Republican, Populist, ...
CO    [Conservative Republican, Republican, Populist...
CT    [nan, Pro-Administration, Federalist, Republic...
DC                               [Republican, Democrat]
DE    [Anti-Administration, nan, Federalist, Republi...
Name: party, dtype: object

In [0]:
unique_parties_by_state.loc['DC'][-1]

'Democrat'

# Debugging groupby operations
A lot of stuff happens during groupby and the following aggregation - here are some methods to get more insight into what is happening at each stage

## Method 1: Iterating

In [0]:
leg.groupby('state')

In [0]:
# leg.groupby('state') is an iterator, i.e. if we use it within a loop
# we get pairs of state and the rows for that state

for item in leg.groupby('state'):
  state, subtable = item
  break  # causes this loop to execute a single time only

In [0]:
state

'AK'

In [0]:
subtable.head(3)

,last_name,first_name,birthday,gender,type,state,party
6619,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6647,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7442,Grigsby,George,1874-12-02,M,rep,AK,NaN


In [0]:
# A different way to achieve the same is using next() and iter():

state, subtable = next(iter(leg.groupby('state')))

In [0]:
state

'AK'

In [0]:
subtable.head(3)

,last_name,first_name,birthday,gender,type,state,party
6619,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6647,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7442,Grigsby,George,1874-12-02,M,rep,AK,NaN


## Method 2: .groups attribute

In [0]:
# the groups attribute is a dictionary with the group keys as keys and
# the corresponding row labels as values
groups = leg.groupby('state').groups

In [0]:
groups['CA']

Int64Index([ 2650,  2742,  2747,  2830,  2832,  2901,  3078,  3096,  3184,
             3221,
            ...
            11830, 11846, 11860, 11896, 11909, 11923, 11929, 11938, 11949,
            11950],
           dtype='int64', length=361)

In [0]:
leg.loc[2650]

last_name      Gilbert
first_name      Edward
birthday           NaN
gender               M
type               rep
state               CA
party         Democrat
Name: 2650, dtype: object

In [0]:
leg.loc[2742]

last_name          Wright
first_name         George
birthday       1816-06-04
gender                  M
type                  rep
state                  CA
party         Independent
Name: 2742, dtype: object

In [0]:
# we can use the value (the row labels) to take a look at the respective subtable
leg.loc[groups['CA']]

,last_name,first_name,birthday,gender,type,state,party
2650,Gilbert,Edward,NaN,M,rep,CA,Democrat
2742,Wright,George,1816-06-04,M,rep,CA,Independent
2747,Frémont,John,1813-01-21,M,sen,CA,Democrat
2830,Marshall,Edward,1821-06-29,M,rep,CA,Democrat
2832,McCorkle,Joseph,1819-06-24,M,rep,CA,Democrat
...,...,...,...,...,...,...,...
11923,Rohrabacher,Dana,1947-06-21,M,rep,CA,Republican
11929,Royce,Edward,1951-10-12,M,rep,CA,Republican
11938,Valadao,David,1977-04-14,M,rep,CA,Republican
11949,Knight,Steve,1966-12-17,M,rep,CA,Republican


## Method 3: .get_group() method

In [0]:
# the .get_group method accepts a group key and returns the subtable corresponding
# to that group key
leg.groupby('state').get_group('TX')

,last_name,first_name,birthday,gender,type,state,party
2568,Pilsbury,Timothy,1789-04-12,M,rep,TX,Democrat
2671,Kaufman,David,1813-12-18,M,rep,TX,Democrat
2814,Howard,Volney,1809-10-22,M,rep,TX,Democrat
2857,Rusk,Thomas,1803-12-05,M,sen,TX,Democrat
2864,Scurry,Richardson,1811-11-11,M,rep,TX,Democrat
...,...,...,...,...,...,...,...
11911,Johnson,Sam,1930-10-11,M,rep,TX,Republican
11919,Poe,Ted,1948-09-10,M,rep,TX,Republican
11931,Sessions,Pete,1955-03-22,M,rep,TX,Republican
11933,Smith,Lamar,1947-11-19,M,rep,TX,Republican


In [0]:
# These methods are useful to understand better what happens at each step
# e.g. here we do first the groupby and then look at the result for Texas

by_state = leg.groupby('state')['last_name'].count()
by_state.head(3)

state
AK     16
AL    206
AR    117
Name: last_name, dtype: int64

In [0]:
by_state.loc['TX']

256

In [0]:
# here we do the count manually on the subtable for Texas
tx_group = leg.groupby('state').get_group('TX')
tx_group.head(3)

,last_name,first_name,birthday,gender,type,state,party
2568,Pilsbury,Timothy,1789-04-12,M,rep,TX,Democrat
2671,Kaufman,David,1813-12-18,M,rep,TX,Democrat
2814,Howard,Volney,1809-10-22,M,rep,TX,Democrat


In [0]:
tx_group['last_name'].count()

256

# Wide transformations

In [0]:
# apply takes a function name or a lambda expression that operates on a subtable
leg.groupby('state').apply(lambda df: df.shape[0]).head(3)

state
AK     16
AL    206
AR    117
dtype: int64

In [0]:
# here we filter on the states that have at least two senators from the independent party
leg.groupby('state').apply(lambda df: df[df['party'] == 'Independent'].shape[0] > 2).head(3)

state
AK    False
AL    False
AR    False
dtype: bool

In [0]:
# other way of doing the same by defining a function

def at_least_two_independents(subtable):
  return subtable[subtable['party'] == 'Independent'].shape[0] > 2

leg.groupby('state').apply(at_least_two_independents).head(3)

state
AK    False
AL    False
AR    False
dtype: bool

# Homework

### Exercise 2.1: Show the number of states with at least 10 female senators

In [0]:
<YOUR CODE>

party
Adams                      99
Adams Democrat              1
American                   44
American Labor              2
Anti Jackson                1
Anti Jacksonian            31
Anti Masonic               27
Anti-Administration         9
Anti-Jacksonian            14
Anti-Lecompton Democrat     7
Name: state, dtype: int64

### Exercise 2.2: Count the average number of senators per state with missing party ('NaN')

In [0]:
<YOUR CODE>

### Exercise 2.3: Show the states with less than 10 senators

In [0]:
<YOUR CODE>

# Correlation

## Standard correlation (Pearson correlation)

In [0]:
# default method for corr is 'Pearson'
leg.corr()
leg.corrwith(leg['age'])

## Kendall's tau

In [0]:
leg.corr(method='kendall')
leg.corrwith(leg['age'], method='kendall')

## Spearman's correlation

In [0]:
leg.corr(method='spearman')
leg.corrwith(leg['age'], method='spearman')

# ANOVA

In [0]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt

# Get data
df = pd.read_csv("https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/difficile.csv")
df.drop('person', axis= 1, inplace= True)

# Recoding value from numeric to string for convenience
df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)
    
# one-way ANOVA
stats.f_oneway(df['libido'][df['dose'] == 'high'], 
             df['libido'][df['dose'] == 'low'],
             df['libido'][df['dose'] == 'placebo'])

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


F_onewayResult(statistic=5.11864406779661, pvalue=0.024694289538222603)